# **\[SOLAR\]** EPT + RGCN (시뮬레이션)

신록예찬  
2023-07-20

In [14]:
%run 0720.py

In [60]:
import itertools
import time

In [18]:
# read dataframe 
df = pd.read_csv('data_eng_230710.csv')

# make y, y_upper, y_period, time, regions 
y = df.loc[:,'Bukchoncheon':'Gyeongju-si'].to_numpy()
yU = df.loc[:,'Bukchoncheon_Upper':'Gyeongju-si_Upper'].to_numpy()
yP = np.divide(y, yU+1e-10)
t = df.loc[:,'date']
regions = list(df.loc[:,'Bukchoncheon':'Gyeongju-si'].columns)

TRAIN_RATIO = 0.8
T,N = len(t),len(regions) 
LEN_TEST = int(np.floor(T*(1-TRAIN_RATIO)))
LEN_TR = T - LEN_TEST

In [72]:
import datetime

In [84]:
method = ['classic','proposed']
lags = [1,2]
filters = [1,2] 
epoch = [1,2] 

model_name = 'RecurrentGCN' ### 모델변경시 수정필요

col = ['model', 'method', 'lags', 
       'nof_filters', 'epoch', 'calculation_time',
       'index_time',
       'node', 'mse']
df = pd.DataFrame(columns=col)


for iteration in range(30):
    for m,l,f,e in itertools.product(method, lags, filters, epoch):
        if m == 'classic':
            t1 = time.time()
            model = RecurrentGCN(node_features=l, filters=f) ### 모델변경시 수정필요
            yhat = rgcn(y, model = model, train_ratio = TRAIN_RATIO, lags=l, epoch=e)
            yhat[yhat < 0]= 0   
            t2 = time.time()
            c = t2-t1
            t = str(datetime.datetime.now())
            _df2 = pd.concat([pd.Series(regions),pd.Series(((y - yhat)[LEN_TR:, :] ** 2).mean(axis=0))],axis=1)
            _df1 = pd.DataFrame([[model_name,m,l,f,e,c,t]]*N)
            _df = pd.concat([_df1,_df2],axis=1).set_axis(col,axis=1)
            df = pd.concat([df,_df])
        else: # proposed 
            t1 = time.time()
            model = RecurrentGCN(node_features=l, filters=f) ### 모델변경시 수정필요
            yUhat = rgcn(yU, model = model, train_ratio = 0.8, lags=l, epoch=e)
            model = RecurrentGCN(node_features=l, filters=f) ### 모델변경시 수정필요
            yPhat = rgcn(yP, model = model, train_ratio = 0.8, lags=l, epoch=e)
            yPhat[yPhat < 0] =0 
            t2 = time.time()
            c = t2-t1
            t = str(datetime.datetime.now())
            _df2 = pd.concat([pd.Series(regions),pd.Series(((y - yhat)[LEN_TR:, :] ** 2).mean(axis=0))],axis=1)
            _df1 = pd.DataFrame([[model_name,m,l,f,e,c,t]]*N)
            _df = pd.concat([_df1,_df2],axis=1).set_axis(col,axis=1)
            df = pd.concat([df,_df])

2/2

In [94]:
df.groupby('index_time').agg({'mse':np.mean}).reset_index().rename({'mse':'mse_mean'},axis=1).merge(df)